In [14]:
import sys
sys.path.append("../bert")
sys.path.append("../..")
import multiprocessing as mp
from typing import Counter
import argparse
import logging
import time
from tqdm import tqdm
from data_structure.question import Question, NewQuestion
import pandas as pd
from tqdm import tqdm
import time
import multiprocessing as mp
from transformers import AutoTokenizer
import math
tokenizer = AutoTokenizer.from_pretrained(
        "microsoft/codebert-base", local_files_only=True)
def gen_feature(tokens, max_length):

        feature = tokenizer(tokens, max_length=max_length,
                                 padding='max_length', return_attention_mask=True,
                                 return_token_type_ids=False, truncation=True,
                                 return_tensors='pt')
        res = {
            "input_ids": feature["input_ids"].flatten(),
            "attention_mask": feature["attention_mask"].flatten()}
        return res


def process_file_to_tensor(file, title_max, text_max, code_max):
    out_dir = "../data/small_tagdc/final_data/"
    dataset = pd.read_pickle(file)
    file_name = "small_tagdc_tensot.pkl"
    q_list = list()
    for question in dataset:
        qid = question.get_qid()
        title = question.get_title()
        title_feature = gen_feature(title, title_max)
        text = question.get_text()
        text_feature = gen_feature(text, text_max)
        code = question.get_code()
        code_feature = gen_feature(code, code_max)
        date = question.get_creation_date()
        tags = question.get_tag()
        q_list.append(NewQuestion(qid, title_feature, text_feature, code_feature, date, tags))
    import pickle
    with open(out_dir+file_name, 'wb') as f:
        pickle.dump(q_list, f)


file = "../data/small_tagdc/final_data/Posts_50000.pkl"
process_file_to_tensor(file, 100, 512, 512)